<center><font size="6"><b>Showcasing new validation strategy</b></font></center>

# Unchanged codes
Unchanged codes are lumped together below.

In [1]:
# preamble, add some code for formatting, not related tto validation strategy

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import set_matplotlib_formats

set_matplotlib_formats('pdf', 'png')
np.set_printoptions(precision=3,suppress=True)
plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble']= r"\usepackage{subdepth}, \usepackage{type1cm}"

In [2]:
# read the 36-descriptor data
df36Descriptor = pd.read_excel('data/ML_data/descriptor_used.xlsx',header=4,index_col=1)

# clean up the column
columns = [df36Descriptor.columns[1]] + df36Descriptor.columns[3: -11].tolist()

newColumns = {}
for ci in columns:
    if ' ' in ci:
        newColumns[ci] = ci.split(' ',1)[0]
    elif '(' in ci:
        newColumns[ci] = ci.split('(',1)[0]
    else:
        newColumns[ci] = ci

dfShortNames = df36Descriptor[columns].rename(columns=newColumns)

# reduce columns to only contain MOF features
shared_descriptor = [col for col in dfShortNames.columns if col in newColumns]
dfMLReduced = dfShortNames[shared_descriptor]

# the MOFs in "dfMLReduced" and adsorption data sets are different, so it is necessary to match the MOFs in two datasets
def datasetMatch(MOFName):
    dfML= dfMLReduced[dfMLReduced['MOF'].isin(MOFName)].drop_duplicates()
    matchedMOFIndex=np.isin(MOFName, dfML['MOF'].values)
    return matchedMOFIndex, dfML

# read flexibility data
flexibilityList=os.listdir('data/flexibility_data/y_data/adsorption_data') # obtain list of csv files for 9 adsorption uptakes
flexivilityData=[]
adsorbateNameList = []

for i, name in enumerate(flexibilityList):
    # read csv files for certain adsorption uptakes
    df = pd.read_csv('data/flexibility_data/y_data/adsorption_data/' + name)
    
    # obtain the rigid value
    rigidValue = np.array(df[df.columns[1]], dtype = float)
    
    # obtain the flexible mean value
    flexValue = np.mean(np.array(df[df.columns[2:]],dtype=float),axis=1)
    
    # obtain the adsorbate label
    label = np.array([name.split("_")[1] for x in range(0,len(flexValue))],dtype=str)
    adsorbateNameList.append(name.split("_")[1])
    
    # stack the rigid value, flexible mean value and the adsorbate label
    singleSet = np.column_stack([rigidValue,flexValue,label])

    if i == 0:
        # obtain the name list of MOFs
        MOFNameTemp = np.array(df[df.columns[0]], dtype = str)
        MOFName = [x.split("_")[0] for x in MOFNameTemp]
        
        # search the MOF name in "dfMLReduced", generating dfML
        matchedMOFIndex, dfML = datasetMatch(MOFName)
        print("The number of MOFs shared by two datasets are: {:d}.".format(dfML.shape[0]))
        
        # generating flexibilityData as "y"
        flexibilityData = singleSet[matchedMOFIndex,:].copy()
    else:
        # concatenate "y"
        flexibilityData = np.concatenate([flexibilityData.copy(),singleSet[matchedMOFIndex,:].copy()])

# manually add adsorbate descriptors

# Mw/gr.mol-1, Tc/K, Pc/bar, ω, Tb/K, Tf/K

adsorbateData=np.array([
    ['xenon',131.293,289.7,58.4,0.008,164.87,161.2], 
    ['butane',58.1,449.8,39.5,0.3,280.1,146.7], 
    ['propene',42.1,436.9,51.7,0.2,254.8,150.6], 
    ['ethane',30.1,381.8,50.3,0.2,184.0,126.2], 
    ['propane',44.1,416.5,44.6,0.2,230.1,136.5], 
    ['CO2',44.0,295.9,71.8,0.2,317.4,204.9], 
    ['ethene',28.054,282.5,51.2,0.089,169.3,228], 
    ['methane',16.04,190.4,46.0,0.011,111.5,91],
    ['krypton',83.798,209.4,55.0,0.005,119.6,115.6]])

adsorbateData.shape
adDf = pd.DataFrame(data=adsorbateData, columns=["adsorbate", "Mw/gr.mol-1", "Tc/K", "Pc/bar", "ω", "Tb/K", "Tf/K"])

# sort the dataframe based on adsorbateNameList
sorterIndex = dict(zip(adsorbateNameList,range(len(adsorbateNameList))))
adDf['an_Rank'] = adDf['adsorbate'].map(sorterIndex)
adDf.sort_values(['an_Rank'],ascending = [True], inplace = True)
adDf.drop('an_Rank', 1, inplace = True)
adDfFloat = adDf.iloc[:, 1:].astype(np.float)
adDfFloat["adsorbate"] = adDf["adsorbate"]


# replicate dfML for 9 adsorbates
dfMLReplicate = pd.concat([dfML]*9)

# replicate adDf for 89 MOFs
adDfReplicate = pd.DataFrame(np.repeat(adDfFloat.values,89,axis=0))
adDfReplicate.columns = adDfFloat.columns

# concatenate two datasets
dfMLReplicate.reset_index(drop=True, inplace=True)
adDfReplicate.reset_index(drop=True, inplace=True)
XAllDescriptor = pd.concat([dfMLReplicate, adDfReplicate],axis=1)

X = np.concatenate((XAllDescriptor.iloc[:, 1:-1], flexibilityData[:, 0].reshape(-1, 1)),axis=1).astype(np.float)
y = flexibilityData[:, 1].astype('float64') .reshape(-1,1)


# feature scaling
X_scaled = (X - X.mean(axis=0))/X.std(axis=0) 

The number of MOFs shared by two datasets are: 89.


# data set split
## Validation set split

In [3]:
np.random.seed(5)
from sklearn.model_selection import train_test_split

# combine the unscaled and scaled X, so that they can be split together
X_combined = np.concatenate((X, X_scaled), axis=1)

# ---------------------------- don't touch the validation set ----------------------------
X_train_test_combined, X_validation_combined, y_train_test, y_validation = train_test_split(X_combined, \
                                                                                            y, test_size=0.25)
X_validation, X_validation_scaled = X_validation_combined[:, :35], X_validation_combined[:, 35:]
# ---------------------------- don't touch the validation set ----------------------------

## 5-fold on train-test set

In [4]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle = True)

for i, (train_index, test_index) in enumerate(kf.split(X_train_test_combined)):
    
    # initialize sets
    if i == 0:
        X_train_combined_5fold = np.zeros(X_train_test_combined[train_index].shape + (5,))
        X_test_combined_5fold = np.zeros(X_train_test_combined[test_index].shape + (5,))
        y_train_5fold = np.zeros(y_train_test[train_index].shape + (5,))
        y_test_5fold = np.zeros(y_train_test[test_index].shape + (5,))
        
    X_train_combined_5fold[:, :, i], X_test_combined_5fold[:, :, i] = X_train_test_combined[train_index], X_train_test_combined[test_index]
    y_train_5fold[:, :, i], y_test_5fold[:, :, i] = y_train_test[train_index], y_train_test[test_index]

X_train_5fold, X_train_scaled_5fold = X_train_combined_5fold[:, :35], X_train_combined_5fold[:, 35:]
X_test_5fold, X_test_scaled_5fold = X_test_combined_5fold[:, :35], X_test_combined_5fold[:, 35:]

# Show how to use 5-fold sets

The code below is only meant for showing how to use 5-fold strategy for training models.
<span style="color:red">They are **not** examples of the whole modeling work</span>.

## Without hyperparameter tuning - using multi-linear regression as example

In [5]:
from sklearn.linear_model import LinearRegression

LR_r2_list = []
LR_coef_list = []

for i in range(5):
    LR_model = LinearRegression()
    LR_model.fit(X_train_5fold[:, :, i], y_train_5fold[:, :, i])
    
    LR_r2_list.append(LR_model.score(X_test_5fold[:, :, i], y_test_5fold[:, :, i]))
    LR_coef_list.append(LR_model.coef_)


best_i = np.array(LR_r2_list).argmax()
LR_coef_best = LR_coef_list[best_i]

for i in range(5):
    if i != best_i:
        print("Fold {}:\tr2={:.3f}".format(i, LR_r2_list[i]))
    else:
        print("Fold {}:\tr2={:.3f}\t<-best performance".format(i, LR_r2_list[i]))

Fold 0:	r2=0.987	<-best performance
Fold 1:	r2=0.980
Fold 2:	r2=0.979
Fold 3:	r2=0.981
Fold 4:	r2=0.962


## With hyperparameter tuning - using rbf kernel regression as example

In [6]:
from sklearn.metrics.pairwise import rbf_kernel

sigmas = np.array([1E-4, 5E-4, 1E-3, 5E-3, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 20, 25, 30, 40, 50, 60])
gammas = 1./(2*sigmas**2)
r2_matrix = np.zeros((gammas.size, 3))

rbf_r2_list = []
rbf_coef_list = []
rbf_gamma_list = []

for i in range(5):
    
    rbf_coef_temp_list = [] # temporarily store parameter values for each gamma
    
    for j, gamma in enumerate(gammas):
    
        X_train_kernel = rbf_kernel(X_train_5fold[:, :, i], X_train_5fold[:, :, i], gamma = gamma)
        X_test_kernel = rbf_kernel(X_test_5fold[:, :, i], X_train_5fold[:, :, i], gamma = gamma)
        model_rbf = LinearRegression()
        model_rbf.fit(X_train_kernel, y_train_5fold[:, :, i])

        r2_matrix[j, 0] = gamma
        r2_matrix[j, 1] = model_rbf.score(X_train_kernel, y_train_5fold[:, :, i])
        r2_matrix[j, -1] = model_rbf.score(X_test_kernel, y_test_5fold[:, :, i])
        
        rbf_coef_temp_list.append(model_rbf.coef_)

    # For each fold, record the optimal gamma and the corresponding parameter values and r2
    n = r2_matrix[:, -1].argmax()
    rbf_r2_list.append(r2_matrix[n, -1])
    rbf_gamma_list.append(r2_matrix[n, 0])
    rbf_coef_list.append(rbf_coef_temp_list[n])


best_i = np.array(rbf_r2_list).argmax()
rbf_gamma_best = rbf_gamma_list[best_i]
rbf_coef_best = rbf_coef_list[best_i]

for i in range(5):
    if i != best_i:
        print("Fold {}:\tr2={:.3f},\toptimal gamma={:.0e}".format(i, rbf_r2_list[i], rbf_gamma_list[i]))
    else:
        print("Fold {}:\tr2={:.3f},\toptimal gamma={:.0e}\t\t<-best performance".format(i, rbf_r2_list[i], rbf_gamma_list[i]))

Fold 0:	r2=0.016,	optimal gamma=2e+02
Fold 1:	r2=0.075,	optimal gamma=5e-01		<-best performance
Fold 2:	r2=0.003,	optimal gamma=5e+03
Fold 3:	r2=0.023,	optimal gamma=5e+01
Fold 4:	r2=0.022,	optimal gamma=2e+02


## Notes

- Again, the code above does not correspond to the entire work for a model. More work can be done, e.g., 
    - detailed analysis of results
    - more sophisticated hyperparameter tuning
    - comparison of using unscaled and scaled data
    - comparison with other models, especially for pure rbf, KRR and LASSO
- To use the scaled X, replace `X_train_5fold`, `X_test_5fold` with `X_train_scaled_5fold`, `X_test_scaled_5fold`
- Remember to record the values of the parameters and the hyperparameters of the best model, as they will be used for future comparison. For example, in "rbf kernel regression" above, I have recorded the optimal hyperparameter in `rbf_gamma_best` and the corresponding optimal parameters in `rbf_coef_best`.

# LASSO Regression - Multi-linear

In [11]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV

sigmas = np.array([1E-10, 1E-8, 1E-5, 1E-3, 0.1, 1, 10, 50, 100, 1000])
gammas = 1./(2*sigmas**2)
alphas = np.array([1E-10, 1E-8, 1E-5, 1E-3, 0.1, 1, 10, 50, 100, 1000])
parameter_ranges = {'alpha':alphas, 'gamma':gammas}

KRR_r2_train_list = []
KRR_r2_test_list = []
KRR_coef_list = []
KRR_gamma_list = []
KRR_alpha_list = []

for i in range(5):
    
    cv = [(slice(None), slice(None))]

    X_train = X_train_5fold[:, :, i]
    X_test = X_test_5fold[:, :, i]
    y_train = y_train_5fold[:, :, i]
    y_test = y_test_5fold[:, :, i]
    
    KRR = KernelRidge(kernel='rbf')
    KRR_search = GridSearchCV(KRR, parameter_ranges, cv=cv)
    KRR_search.fit(X_train,y_train)
 
    KRR_test = KernelRidge(alpha=KRR_search.best_estimator_.alpha, kernel='rbf', gamma=KRR_search.best_estimator_.gamma)
    KRR_test = KRR_search.best_estimator_
    print(KRR_test.score(X_test, y_test))
    
    KRR_r2_train_list.append(KRR_search.best_score_)
    KRR_r2_test_list.append(KRR_test.score(X_test,y_test))
    KRR_coef_list.append(KRR_test.dual_coef_)
    KRR_gamma_list.append(KRR_search.best_estimator_.gamma)
    KRR_alpha_list.append(KRR_search.best_estimator_.alpha)
    
best_i = np.array(KRR_r2_test_list).argmax()
KRR_gamma_best = KRR_gamma_list[best_i]
KRR_alpha_best = KRR_alpha_list[best_i]
KRR_coef_best = KRR_coef_list[best_i]

for i in range(5):
    if i != best_i:
        print("Fold {}:\tr2={:.10f},\toptimal gamma={:.0e},\toptimal alpha={:.0e}".format(i, KRR_r2_test_list[i], KRR_gamma_list[i], KRR_alpha_list[i]))
    else:
        print("Fold {}:\tr2={:.10f},\toptimal gamma={:.0e},\toptimal alpha={:.0e}\t\t<-best performance".format(i, KRR_r2_test_list[i], KRR_gamma_list[i], KRR_alpha_list[i]))

-1.0829579778177312
-1.081001905203864
-1.2235054972002368
-1.4946730511734123
-1.462506305738168
Fold 0:	r2=-1.0829579778,	optimal gamma=5e+01,	optimal alpha=1e-10
Fold 1:	r2=-1.0810019052,	optimal gamma=5e+01,	optimal alpha=1e-10		<-best performance
Fold 2:	r2=-1.2235054972,	optimal gamma=5e+01,	optimal alpha=1e-10
Fold 3:	r2=-1.4946730512,	optimal gamma=5e+01,	optimal alpha=1e-10
Fold 4:	r2=-1.4625063057,	optimal gamma=5e+01,	optimal alpha=1e-10


# LASSO Regression with RBF Kernel

In [12]:
float("inf") - 1

inf